### retrieve and preprocess data for the webapp
In this notebook data from the SpaceDevs LaunchLibrary2 API is extracted and prepared for usage in the webapp

In [28]:
import json
import requests
import pandas as pd
import time

In [29]:
API_URL = "https://ll.thespacedevs.com/2.2.0"
LAUNCHER_CONFIG_ENDPOINT = "/config/launcher"


PARAMS = {
    "limit": 100,
    "active": True,
    "successful_launches__gte": 10
}


FINAL_URL = API_URL + LAUNCHER_CONFIG_ENDPOINT


In [33]:
vehicles = requests.get(FINAL_URL, params=PARAMS)

In [34]:
vehicles

<Response [200]>

In [35]:
vehicles.json()["results"]

[{'id': 16,
  'url': 'https://ll.thespacedevs.com/2.2.0/config/launcher/16/',
  'name': 'Atlas V 401',
  'manufacturer': {'id': 124,
   'url': 'https://ll.thespacedevs.com/2.2.0/agencies/124/',
   'name': 'United Launch Alliance',
   'featured': True,
   'type': 'Commercial',
   'country_code': 'USA',
   'abbrev': 'ULA',
   'description': 'United Launch Alliance (ULA) is a joint venture of Lockheed Martin Space Systems and Boeing Defense, Space & Security. ULA was formed in December 2006 by combining the teams at these companies which provide spacecraft launch services to the government of the United States. ULA launches from both coasts of the US. They launch their Atlas V vehicle from LC-41 in Cape Canaveral and LC-3E at Vandeberg. Their Delta IV launches from LC-37 at Cape Canaveral and LC-6 at Vandenberg.',
   'administrator': 'CEO: Tory Bruno',
   'founding_year': '2006',
   'launchers': 'Atlas | Delta IV | Vulcan',
   'spacecraft': 'CST-100 Starliner',
   'parent': None,
   'imag

In [36]:
LAUNCHER_URLS = []
for vehicle in vehicles.json()["results"]:
    print(vehicle)

{'id': 16, 'url': 'https://ll.thespacedevs.com/2.2.0/config/launcher/16/', 'name': 'Atlas V 401', 'manufacturer': {'id': 124, 'url': 'https://ll.thespacedevs.com/2.2.0/agencies/124/', 'name': 'United Launch Alliance', 'featured': True, 'type': 'Commercial', 'country_code': 'USA', 'abbrev': 'ULA', 'description': 'United Launch Alliance (ULA) is a joint venture of Lockheed Martin Space Systems and Boeing Defense, Space & Security. ULA was formed in December 2006 by combining the teams at these companies which provide spacecraft launch services to the government of the United States. ULA launches from both coasts of the US. They launch their Atlas V vehicle from LC-41 in Cape Canaveral and LC-3E at Vandeberg. Their Delta IV launches from LC-37 at Cape Canaveral and LC-6 at Vandenberg.', 'administrator': 'CEO: Tory Bruno', 'founding_year': '2006', 'launchers': 'Atlas | Delta IV | Vulcan', 'spacecraft': 'CST-100 Starliner', 'parent': None, 'image_url': 'https://spacelaunchnow-prod-east.nyc3

In [37]:
pd.DataFrame(vehicles.json()["results"]).loc[:0]

,id,url,name,manufacturer,program,family,full_name,variant,reusable,image_url,info_url,wiki_url
0,16,https://ll.thespacedevs.com/2.2.0/config/launc...,Atlas V 401,"{'id': 124, 'url': 'https://ll.thespacedevs.co...",[],Atlas,Atlas V 401,401,False,https://spacelaunchnow-prod-east.nyc3.digitalo...,https://www.ulalaunch.com/rockets/atlas-v,https://en.wikipedia.org/wiki/Atlas_V


In [38]:
launcher_urls = [vehicle["url"] for vehicle in vehicles.json()["results"]]

In [39]:
launcher_details = []
for (count, url) in enumerate(launcher_urls):
    curr_launcher = requests.get(url)
    while curr_launcher.status_code == 429:
        print(f"Failed fetching launcher #{count}")
        time.sleep(4*60+10) # sleep 4:10 minutes (rate limit)
        curr_launcher = requests.get(url)
    launcher_details.append(curr_launcher) # append new launcher to list
    print(f"Successfully fetched launcher #{count}")


In [43]:
launcher_details[0].json()

{'id': 16,
 'url': 'https://ll.thespacedevs.com/2.2.0/config/launcher/16/',
 'name': 'Atlas V 401',
 'active': True,
 'reusable': False,
 'description': 'An Atlas V with a 4 meter diameter fairing, no boosters and a single centaur upper stage engine.',
 'family': 'Atlas',
 'full_name': 'Atlas V 401',
 'manufacturer': {'id': 124,
  'url': 'https://ll.thespacedevs.com/2.2.0/agencies/124/',
  'name': 'United Launch Alliance',
  'featured': True,
  'type': 'Commercial',
  'country_code': 'USA',
  'abbrev': 'ULA',
  'description': 'United Launch Alliance (ULA) is a joint venture of Lockheed Martin Space Systems and Boeing Defense, Space & Security. ULA was formed in December 2006 by combining the teams at these companies which provide spacecraft launch services to the government of the United States. ULA launches from both coasts of the US. They launch their Atlas V vehicle from LC-41 in Cape Canaveral and LC-3E at Vandeberg. Their Delta IV launches from LC-37 at Cape Canaveral and LC-6 at 

In [44]:
json_objects = [launcher.json() for launcher in launcher_details]

In [47]:
with open("launchers_raw.json", 'w') as file:
    json.dump(json_objects, file, indent=4)